In [104]:
import pandas as pd
import math
import Vector_similarity

pd.set_option('display.max_columns', None) # dataframe 잘림없이 출력
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None) # 리스트 잘림없이 출력
pd.options.display.float_format = '{:.5f}'.format # e, 지수없이 출력

In [105]:
df = pd.read_csv('./nvidiasmi_devicequery_all.csv')
df.head(2)

,Train_Name,Instances,model,batchsize,datasize,totaldata,total_latency_second,one_epoch_average_second,clocks_current_graphicsMHz_mean,clocks_current_graphicsMHz_std,clocks_current_graphicsMHz_min,clocks_current_graphicsMHz_25,clocks_current_graphicsMHz_50,clocks_current_graphicsMHz_75,clocks_current_graphicsMHz_max,clocks_current_smMHz_mean,clocks_current_smMHz_std,clocks_current_smMHz_min,clocks_current_smMHz_25,clocks_current_smMHz_50,clocks_current_smMHz_75,clocks_current_smMHz_max,memory_freeMiB_mean,memory_freeMiB_std,memory_freeMiB_min,memory_freeMiB_25,memory_freeMiB_50,memory_freeMiB_75,memory_freeMiB_max,memory_totalMiB_mean,memory_totalMiB_std,memory_totalMiB_min,memory_totalMiB_25,memory_totalMiB_50,memory_totalMiB_75,memory_totalMiB_max,memory_usedMiB_mean,memory_usedMiB_std,memory_usedMiB_min,memory_usedMiB_25,memory_usedMiB_50,memory_usedMiB_75,memory_usedMiB_max,power_drawW_mean,power_drawW_std,power_drawW_min,power_drawW_25,power_drawW_50,power_drawW_75,power_drawW_max,temperature_gpu_mean,temperature_gpu_std,temperature_gpu_min,temperature_gpu_25,temperature_gpu_50,temperature_gpu_75,temperature_gpu_max,utilization_gpu_mean,utilization_gpu_std,utilization_gpu_min,utilization_gpu_25,utilization_gpu_50,utilization_gpu_75,utilization_gpu_max,utilization_memory_mean,utilization_memory_std,utilization_memory_min,utilization_memory_25,utilization_memory_50,utilization_memory_75,utilization_memory_max,accounting_buffer_size,accounting_mode,clocks_applications_graphicsMHz,clocks_applications_memoryMHz,clocks_current_memoryMHz,clocks_default_applications_graphicsMHz,clocks_default_applications_memoryMHz,clocks_max_graphicsMHz,clocks_max_memoryMHz,clocks_max_smMHz,clocks_throttle_reasons_applications_clocks_setting,clocks_throttle_reasons_gpu_idle,clocks_throttle_reasons_hw_slowdown,clocks_throttle_reasons_supported,compute_mode,display_active,display_mode,driver_model_current,driver_model_pending,driver_version,fan_speed,gom_current,gom_pending,index,inforom_ecc,inforom_img,inforom_oem,inforom_pwr,memory_totalMiB,name,new_timestamp_gpu,pci_bus,pci_bus_id,pci_device,pci_device_id,pci_domain,pci_sub_device_id,pcie_link_gen_current,pcie_link_gen_max,pcie_link_width_current,pcie_link_width_max,persistence_mode,power_default_limitW,power_limitW,power_management,power_max_limitW,power_min_limitW,pstate,serial,uuid,vbios_version,Total amount of global memory(MBytes),Multiprocessors,CUDA Cores/MP,CUDA Cores,GPU Max Clock rate(MHz),Memory Clock rate,Memory Bus Width,L2 Cache Size,Total shared memory per multiprocessor,Maximum number of threads per multiprocessor,Concurrent copy and kernel execution copy engines,Memory Bandwidth
0,p100_MNIST_CNN_batchsize32_datasize224_total_e...,p100,MNIST_CNN,32,224,2000,28.23200,2.82320,1328.00000,0.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,0.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,213.28226,56.67723,199.00000,199.00000,199.00000,199.00000,499.00000,16280.00000,0.00000,16280.00000,16280.00000,16280.00000,16280.00000,16280.00000,16066.71774,56.67723,15781.00000,16081.00000,16081.00000,16081.00000,16081.00000,122.11242,31.98677,40.51000,117.35000,130.34000,142.26500,161.27000,53.40323,0.93445,51.00000,53.00000,54.00000,54.00000,54.00000,76.42742,25.45272,0.00000,81.00000,87.00000,89.00000,91.00000,39.08065,15.48127,0.00000,38.00000,47.00000,48.00000,49.00000,4000,Disabled,1189.00000,715.00000,715.00000,1189,715,1328.00000,715.00000,1328.00000,NotActive,NotActive,NotActive,0x00000000000001FF,Default,Disabled,Enabled,[N/A],[N/A],470.57.02,[N/A],[N/A],[N/A],0,4.10000,H400.0201.00.08,1.10000,[N/A],16280.00000,TeslaP100-PCIE-16GB,20220411012115.12109,0x00,00000000:00:07.0,0x07,0x15F810DE,0x0000,0x118F10DE,3,3,16,16,Enabled,250.00000,250.00000,Enabled,250.00000,125.00000,P0,325017077873,GPU-06c7b23d-6a59-60a6-78ee-82310fd08348,86.00.41.00.06,16280.90000,56,64,3584,1328.50000,715,4096,4194304,65536,2048,2,3592192
1,p100_MNIST_CNN_batchsize16_datasize

onehot encoding

In [106]:
# for i in list(set(df.columns)-set(df.describe().columns)):   # 원핫인코딩 추가로 해야할것이 있는지 => pci_bus_id / pci_device 
#     print(i)
#     print(df[i].value_counts())
#     print(" ")    

In [107]:
df = pd.get_dummies(data = df, columns = ['pci_bus_id'], prefix = 'pci_bus_id')
df = pd.get_dummies(data = df, columns = ['pci_device'], prefix = 'pci_device')

컬럼들 거르기(시리얼넘버나 version같은...)

In [108]:
vector_features = df.describe().columns

In [109]:
df[vector_features].head(2)
# 빼주어야할 항목 : batchsize /datasize /totaldata /total_latency_second /one_epoch_average_second / index /inforom_ecc /inforom_oem/new_timestamp_gpu /serial

,batchsize,datasize,totaldata,total_latency_second,one_epoch_average_second,clocks_current_graphicsMHz_mean,clocks_current_graphicsMHz_std,clocks_current_graphicsMHz_min,clocks_current_graphicsMHz_25,clocks_current_graphicsMHz_50,clocks_current_graphicsMHz_75,clocks_current_graphicsMHz_max,clocks_current_smMHz_mean,clocks_current_smMHz_std,clocks_current_smMHz_min,clocks_current_smMHz_25,clocks_current_smMHz_50,clocks_current_smMHz_75,clocks_current_smMHz_max,memory_freeMiB_mean,memory_freeMiB_std,memory_freeMiB_min,memory_freeMiB_25,memory_freeMiB_50,memory_freeMiB_75,memory_freeMiB_max,memory_totalMiB_mean,memory_totalMiB_std,memory_totalMiB_min,memory_totalMiB_25,memory_totalMiB_50,memory_totalMiB_75,memory_totalMiB_max,memory_usedMiB_mean,memory_usedMiB_std,memory_usedMiB_min,memory_usedMiB_25,memory_usedMiB_50,memory_usedMiB_75,memory_usedMiB_max,power_drawW_mean,power_drawW_std,power_drawW_min,power_drawW_25,power_drawW_50,power_drawW_75,power_drawW_max,temperature_gpu_mean,temperature_gpu_std,temperature_gpu_min,temperature_gpu_25,temperature_gpu_50,temperature_gpu_75,temperature_gpu_max,utilization_gpu_mean,utilization_gpu_std,utilization_gpu_min,utilization_gpu_25,utilization_gpu_50,utilization_gpu_75,utilization_gpu_max,utilization_memory_mean,utilization_memory_std,utilization_memory_min,utilization_memory_25,utilization_memory_50,utilization_memory_75,utilization_memory_max,accounting_buffer_size,clocks_applications_graphicsMHz,clocks_applications_memoryMHz,clocks_current_memoryMHz,clocks_default_applications_graphicsMHz,clocks_default_applications_memoryMHz,clocks_max_graphicsMHz,clocks_max_memoryMHz,clocks_max_smMHz,index,inforom_ecc,inforom_oem,memory_totalMiB,new_timestamp_gpu,pcie_link_gen_current,pcie_link_gen_max,pcie_link_width_current,pcie_link_width_max,power_default_limitW,power_limitW,power_max_limitW,power_min_limitW,serial,Total amount of global memory(MBytes),Multiprocessors,CUDA Cores/MP,CUDA Cores,GPU Max Clock rate(MHz),Memory Clock rate,Memory Bus Width,L2 Cache Size,Total shared memory per multiprocessor,Maximum number of threads per multiprocessor,Concurrent copy and kernel execution copy engines,Memory Bandwidth,pci_bus_id_00000000:00:07.0,pci_bus_id_00000000:00:1E.0,pci_device_0x07,pci_device_0x1E
0,32,224,2000,28.23200,2.82320,1328.00000,0.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,0.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,213.28226,56.67723,199.00000,199.00000,199.00000,199.00000,499.00000,16280.00000,0.00000,16280.00000,16280.00000,16280.00000,16280.00000,16280.00000,16066.71774,56.67723,15781.00000,16081.00000,16081.00000,16081.00000,16081.00000,122.11242,31.98677,40.51000,117.35000,130.34000,142.26500,161.27000,53.40323,0.93445,51.00000,53.00000,54.00000,54.00000,54.00000,76.42742,25.45272,0.00000,81.00000,87.00000,89.00000,91.00000,39.08065,15.48127,0.00000,38.00000,47.00000,48.00000,49.00000,4000,1189.00000,715.00000,715.00000,1189,715,1328.00000,715.00000,1328.00000,0,4.10000,1.10000,16280.00000,20220411012115.12109,3,3,16,16,250.00000,250.00000,250.00000,125.00000,325017077873,16280.90000,56,64,3584,1328.50000,715,4096,4194304,65536,2048,2,3592192,1,0,1,0
1,16,224,2000,38.14100,3.81410,1328.00000,0.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,0.00000,1328.00000,1328.00000,1328.00000,1328.00000,1328.00000,210.07101,49.53745,199.00000,199.00000,199.00000,199.00000,499.00000,16280.00000,0.00000,16280.00000,16280.00000,16280.00000,16280.00000,16280.00000,16069.92899,49.53745,15781.00000,16081.00000,16081.00000,16081.00000,16081.00000,120.99642,27.34047,41.00000,116.65250,128.91500,139.28500,148.50000,53.43195,0.92594,50.00000,53.00000,54.00000,54.00000,54.00000,79.48817,23.10141,0.00000,83.00000,89.00000,90.00000,92.00000,41.91420,15.05899,0.00000,45.00000,49.00000,50.00000,51.00000,4000,1189.00000,715.00000,715.00000,1189,715,1328.00000,715.00000,1328.00000,0,4.10000,1.10000,16280.00000,20220411012115.12109,3,3

In [97]:
vector_features = list(set(vector_features)-set(['batchsize','datasize','totaldata','total_latency_second','one_epoch_average_second','index','inforom_ecc','inforom_oem','new_timestamp_gpu','serial'])) 

각 train별로 similiarity 구하기(use_all_vector_features)

In [98]:
Name_list = list(df["Train_Name"])
DataFrame_Cosine = pd.DataFrame(index=Name_list,columns=Name_list)
DataFrame_Euclidean = pd.DataFrame(index=Name_list,columns=Name_list)
DataFrame_Triangle = pd.DataFrame(index=Name_list,columns=Name_list)
DataFrame_Sector = pd.DataFrame(index=Name_list,columns=Name_list)
DataFrame_TS_SS = pd.DataFrame(index=Name_list,columns=Name_list)

In [99]:
for name in Name_list:
    print(name)
    main_vector = list((df[df['Train_Name'] == name][vector_features].values)[0])
    for compare_name in Name_list:
        if(name == compare_name):
            DataFrame_Cosine[name][compare_name] = 500000
            DataFrame_Euclidean[name][compare_name] = 500000
            DataFrame_Triangle[name][compare_name] = 500000
            DataFrame_Sector[name][compare_name] = 500000
            DataFrame_TS_SS[name][compare_name] = 500000
        else:
            compare_vector = list((df[df['Train_Name'] == compare_name][vector_features].values)[0]) 
            DataFrame_Cosine[name][compare_name] = Vector_similarity.Cosine(main_vector,compare_vector)
            DataFrame_Euclidean[name][compare_name] = Vector_similarity.Euclidean(main_vector,compare_vector)
            DataFrame_Triangle[name][compare_name] = Vector_similarity.Triangle(main_vector,compare_vector)
            DataFrame_Sector[name][compare_name] = Vector_similarity.Sector(main_vector,compare_vector)
            DataFrame_TS_SS[name][compare_name] = Vector_similarity.TS_SS(main_vector,compare_vector)

p100_MNIST_CNN_batchsize32_datasize224_total_epoch10_totaldata2000
p100_MNIST_CNN_batchsize16_datasize224_total_epoch10_totaldata2000
p100_MNIST_CNN_batchsize64_datasize128_total_epoch10_totaldata5000
p100_MNIST_CNN_batchsize256_datasize128_total_epoch10_totaldata5000
p100_MNIST_CNN_batchsize32_datasize64_total_epoch10_totaldata5000
p100_MNIST_CNN_batchsize16_datasize64_total_epoch10_totaldata5000
p100_MNIST_CNN_batchsize128_datasize224_total_epoch10_totaldata2000
p100_MNIST_CNN_batchsize64_datasize224_total_epoch10_totaldata2000
p100_MNIST_CNN_batchsize32_datasize128_total_epoch10_totaldata5000
p100_MNIST_CNN_batchsize16_datasize128_total_epoch10_totaldata5000
p100_MNIST_CNN_batchsize256_datasize224_total_epoch10_totaldata2000
p100_MNIST_CNN_batchsize16_datasize32_total_epoch10_totaldata50000
p100_MNIST_CNN_batchsize256_datasize256_total_epoch10_totaldata2000
p100_MNIST_CNN_batchsize64_datasize32_total_epoch10_totaldata50000
p100_MNIST_CNN_batchsize128_datasize64_total_epoch10_totalda

p100_ResNet50_batchsize16_datasize256_total_epoch10_totaldata2000
p100_ResNet50_batchsize16_datasize128_total_epoch10_totaldata5000
p100_ResNet50_batchsize128_datasize224_total_epoch10_totaldata2000
p100_ResNet50_batchsize64_datasize256_total_epoch10_totaldata2000
p100_ResNet50_batchsize32_datasize224_total_epoch10_totaldata2000
p100_ResNet50_batchsize32_datasize64_total_epoch10_totaldata5000
p100_ResNet50_batchsize32_datasize256_total_epoch10_totaldata2000
p100_InceptionResNetV2_batchsize64_datasize256_total_epoch10_totaldata2000
p100_InceptionResNetV2_batchsize128_datasize128_total_epoch10_totaldata5000
p100_InceptionResNetV2_batchsize64_datasize128_total_epoch10_totaldata5000
p100_InceptionResNetV2_batchsize64_datasize224_total_epoch10_totaldata2000
p100_InceptionResNetV2_batchsize16_datasize256_total_epoch10_totaldata2000
p100_InceptionResNetV2_batchsize16_datasize224_total_epoch10_totaldata2000
p100_InceptionResNetV2_batchsize16_datasize128_total_epoch10_totaldata5000
p100_Incepti

p3_CIFAR10_CNN_batchsize32_datasize256_total_epoch10_totaldata2000
p3_CIFAR10_CNN_batchsize128_datasize128_total_epoch10_totaldata5000
p3_CIFAR10_CNN_batchsize64_datasize64_total_epoch10_totaldata5000
p3_CIFAR10_CNN_batchsize256_datasize256_total_epoch10_totaldata2000
p3_CIFAR10_CNN_batchsize256_datasize32_total_epoch10_totaldata50000
p3_CIFAR10_CNN_batchsize128_datasize224_total_epoch10_totaldata2000
p3_CIFAR10_CNN_batchsize64_datasize256_total_epoch10_totaldata2000
p3_CIFAR10_CNN_batchsize256_datasize128_total_epoch10_totaldata5000
p3_CIFAR10_CNN_batchsize16_datasize224_total_epoch10_totaldata2000
p3_CIFAR10_CNN_batchsize256_datasize224_total_epoch10_totaldata2000
p3_CIFAR10_CNN_batchsize64_datasize32_total_epoch10_totaldata50000
p3_CIFAR10_CNN_batchsize32_datasize64_total_epoch10_totaldata5000
p3_CIFAR10_CNN_batchsize16_datasize256_total_epoch10_totaldata2000
p3_CIFAR10_CNN_batchsize16_datasize32_total_epoch10_totaldata50000
p3_CIFAR10_CNN_batchsize64_datasize128_total_epoch10_total

p3_ResNetSmall_batchsize128_datasize256_total_epoch10_totaldata2000
p3_ResNetSmall_batchsize128_datasize128_total_epoch10_totaldata5000
p3_ResNetSmall_batchsize32_datasize128_total_epoch10_totaldata5000
p3_ResNetSmall_batchsize16_datasize64_total_epoch10_totaldata5000
p3_ResNetSmall_batchsize16_datasize32_total_epoch10_totaldata50000
p3_ResNetSmall_batchsize256_datasize256_total_epoch10_totaldata2000
p3_ResNetSmall_batchsize256_datasize224_total_epoch10_totaldata2000
p3_ResNetSmall_batchsize128_datasize64_total_epoch10_totaldata5000
p3_ResNetSmall_batchsize16_datasize224_total_epoch10_totaldata2000
p3_ResNetSmall_batchsize32_datasize256_total_epoch10_totaldata2000
p3_ResNetSmall_batchsize128_datasize32_total_epoch10_totaldata50000
g4dn_MNIST_CNN_batchsize32_datasize224_total_epoch10_totaldata2000
g4dn_MNIST_CNN_batchsize16_datasize224_total_epoch10_totaldata2000
g4dn_MNIST_CNN_batchsize64_datasize128_total_epoch10_totaldata5000
g4dn_MNIST_CNN_batchsize256_datasize128_total_epoch10_tota

g4dn_LeNet5_batchsize16_datasize64_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize64_datasize128_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize32_datasize224_total_epoch10_totaldata2000
g4dn_ResNetSmall_batchsize16_datasize128_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize64_datasize32_total_epoch10_totaldata50000
g4dn_ResNetSmall_batchsize256_datasize64_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize256_datasize32_total_epoch10_totaldata50000
g4dn_ResNetSmall_batchsize64_datasize64_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize32_datasize64_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize32_datasize32_total_epoch10_totaldata50000
g4dn_ResNetSmall_batchsize256_datasize128_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize128_datasize128_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize32_datasize128_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize16_datasize64_total_epoch10_totaldata5000
g4dn_ResNetSmall_batchsize16_datasize32_

g5_VGG19_batchsize64_datasize256_total_epoch10_totaldata2000
g5_VGG19_batchsize128_datasize64_total_epoch10_totaldata5000
g5_ResNet50_batchsize128_datasize256_total_epoch10_totaldata2000
g5_ResNet50_batchsize16_datasize64_total_epoch10_totaldata5000
g5_ResNet50_batchsize64_datasize128_total_epoch10_totaldata5000
g5_ResNet50_batchsize128_datasize64_total_epoch10_totaldata5000
g5_ResNet50_batchsize16_datasize224_total_epoch10_totaldata2000
g5_ResNet50_batchsize32_datasize32_total_epoch10_totaldata50000
g5_ResNet50_batchsize128_datasize32_total_epoch10_totaldata50000
g5_ResNet50_batchsize64_datasize64_total_epoch10_totaldata5000
g5_ResNet50_batchsize256_datasize128_total_epoch10_totaldata5000
g5_ResNet50_batchsize64_datasize224_total_epoch10_totaldata2000
g5_ResNet50_batchsize64_datasize32_total_epoch10_totaldata50000
g5_ResNet50_batchsize128_datasize128_total_epoch10_totaldata5000
g5_ResNet50_batchsize256_datasize32_total_epoch10_totaldata50000
g5_ResNet50_batchsize32_datasize128_total_e

In [102]:
DataFrame_Euclidean.head()

,p100_MNIST_CNN_batchsize32_datasize224_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize16_datasize224_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize64_datasize128_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize256_datasize128_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize32_datasize64_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize16_datasize64_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize128_datasize224_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize64_datasize224_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize32_datasize128_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize16_datasize128_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize256_datasize224_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize16_datasize32_total_epoch10_totaldata50000,p100_MNIST_CNN_batchsize256_datasize256_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize64_datasize32_total_epoch10_totaldata50000,p100_MNIST_CNN_batchsize128_datasize64_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize256_datasize64_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize256_datasize32_total_epoch10_totaldata50000,p100_MNIST_CNN_batchsize64_datasize256_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize128_datasize128_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize32_datasize256_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize16_datasize256_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize64_datasize64_total_epoch10_totaldata5000,p100_MNIST_CNN_batchsize128_datasize256_total_epoch10_totaldata2000,p100_MNIST_CNN_batchsize128_datasize32_total_epoch10_totaldata50000,p100_MNIST_CNN_batchsize32_datasize32_total_epoch10_totaldata50000,p100_AlexNet_batchsize32_datasize256_total_epoch10_totaldata2000,p100_AlexNet_batchsize128_datasize128_total_epoch10_totaldata5000,p100_AlexNet_batchsize16_datasize256_total_epoch10_totaldata2000,p100_AlexNet_batchsize64_datasize32_total_epoch10_totaldata50000,p100_AlexNet_batchsize32_datasize64_total_epoch10_totaldata5000,p100_AlexNet_batchsize16_datasize128_total_epoch10_totaldata5000,p100_AlexNet_batchsize256_datasize32_total_epoch10_totaldata50000,p100_AlexNet_batchsize128_datasize32_total_epoch10_totaldata50000,p100_AlexNet_batchsize64_datasize128_total_epoch10_totaldata5000,p100_AlexNet_batchsize256_datasize128_total_epoch10_totaldata5000,p100_AlexNet_batchsize16_datasize32_total_epoch10_totaldata50000,p100_AlexNet_batchsize256_datasize64_total_epoch10_totaldata5000,p100_AlexNet_batchsize32_datasize32_total_epoch10_totaldata50000,p100_AlexNet_batchsize64_datasize224_total_epoch10_totaldata2000,p100_AlexNet_batchsize16_datasize224_total_epoch10_totaldata2000,p100_AlexNet_batchsize128_datasize64_total_epoch10_totaldata5000,p100_AlexNet_batchsize16_datasize64_total_epoch10_totaldata5000,p100_AlexNet_batchsize64_datasize64_total_epoch10_totaldata5000,p100_AlexNet_batchsize32_datasize224_total_epoch10_totaldata2000,p100_AlexNet_batchsize32_datasize128_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize32_datasize128_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize32_datasize32_total_epoch10_totaldata50000,p100_CIFAR10_CNN_batchsize128_datasize32_total_epoch10_totaldata50000,p100_CIFAR10_CNN_batchsize16_datasize128_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize128_datasize64_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize64_datasize224_total_epoch10_totaldata2000,p100_CIFAR10_CNN_batchsize16_datasize64_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize32_datasize224_total_epoch10_totaldata2000,p100_CIFAR10_CNN_batchsize128_datasize256_total_epoch10_totaldata2000,p100_CIFAR10_CNN_batchsize256_datasize64_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize32_datasize256_total_epoch10_totaldata2000,p100_CIFAR10_CNN_batchsize128_datasize128_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize64_datasize64_total_epoch10_totaldata5000,p100_CIFAR10_CNN_batchsize256_datasize256_total_epoch10_totaldata2000,p100_CIFAR10_CNN_batchsize256_datasize32_total_epo

In [101]:
DataFrame_TS_SS.to_csv("Similiarity_all_TS_SS.csv")
DataFrame_Cosine.to_csv("Similiarity_all_Cosine.csv")
DataFrame_Euclidean.to_csv("Similiarity_all_Euclidean.csv")
DataFrame_Triangle.to_csv("Similiarity_all_Triangle.csv")
DataFrame_Sector.to_csv("Similiarity_all_Sector.csv")